### Criando a conexão

In [1]:
import psycopg2 #biblioteca para conectar ao banco
import pandas as pd #biblioteca para criação e gerenciamento de data frame. Também utilizada para exportar o dataframe para csv
import pandas.io.sql as sqlio #utilizado para integrar o sql com pandas, permitindo exportar os dados direto do banco para um dataframe

In [9]:
conn = psycopg2.connect(    
    database="conectcar",
    user="raul_correa", 
    password="Conect@091!",
    host="sdlf-cntcar-redshift-dev.capkkn406tig.us-east-2.redshift.amazonaws.com",
    port="5439"
)

### Bases Habilitadas - fotos por período

#### *fotos mensal 09 meses atrás*

In [10]:
#version 3.0
cte_BasePF_Clientes1Placa_09 = """"
DROP TABLE IF EXISTS #MesRef_09;
CREATE TABLE #MesRef_09 (MesRefVar DATE);
INSERT INTO #MesRef_09
select TO_DATE(ADD_MONTHS(current_date, -9), 'YYYY-MM-01') as MesRefVar
;

DROP TABLE IF EXISTS #baseHabilitada_09;
CREATE TABLE #baseHabilitada_09 (documento VARCHAR, anomes DATE, agregado VARCHAR, anomesCancelamento DATE);
INSERT INTO #baseHabilitada_09
select distinct
d.documento*1 :: float as documento,
TO_DATE(datahabilitada, 'YYYY-MM-01') as anomes,
a.agregado as agregado,
TO_DATE(b.datacancelamento, 'YYYY-MM-01') as anomesCancelamento
from cntcar_work.vw_redshift_ciclodevida_basehabilitada_stage as a
left join cntcar_work.tb_conectcar_dbo_adesao_stage as b on a.adesaoid = b.adesaoid
left join cntcar_work.tb_conectcar_dbo_veiculo_stage as c on b.veiculoID = c.veiculo_id
left join conectcar.cntcar_work.tb_conectcar_dbo_cliente_stage as d on b.Clienteid = d.Clienteid
where anomes = (SELECT MesRefVar FROM #MesRef_09)
and len(documento) <= 11
;

DROP TABLE IF EXISTS #flagPlacasas_09;
CREATE TABLE #flagPlacasas_09 (documento VARCHAR, flagPlacas FLOAT);
INSERT INTO #flagPlacasas_09
select
documento,
count (documento) as flagPlacas
from #baseHabilitada_09
group by documento
having flagPlacas > 1
order by flagPlacas desc
;

DROP TABLE IF EXISTS #baseHabilitada_09PFCom1PlacaSemPorto;
CREATE TABLE #baseHabilitada_09PFCom1PlacaSemPorto (documento VARCHAR, anomes DATE, agregado VARCHAR, anomescancelamento DATE);
INSERT INTO #baseHabilitada_09PFCom1PlacaSemPorto
select
a.documento ::float,
a.anomes,
a.agregado,
a.anomescancelamento
from #baseHabilitada_09 a
left join #flagPlacasas_09 b
on a.documento = b.documento
where b.flagPlacas is NULL
;

DROP TABLE IF EXISTS #baseHabilitada_09Porto;
CREATE TABLE #baseHabilitada_09Porto (documento VARCHAR, anomes DATE, agregado VARCHAR, anomescancelamento DATE);
INSERT INTO #baseHabilitada_09Porto
select
regexp_replace(c.codigointernoparceiro,'[^0-9]+', '') as Documento,
TO_DATE(datahabilitada, 'YYYY-MM-01') as anomes,
p.agregado,
TO_DATE(a.datacancelamento, 'YYYY-MM-01') as anomescancelamento
from cntcar_work.vw_redshift_ciclodevida_basehabilitada_stage as bh
left outer join cntcar_work.tb_parceiro_dbo_ativacaocontrato_stage ac on ac.adesaoid = bh.adesaoid
left outer join cntcar_work.tb_parceiro_dbo_contrato_stage c on c.contratoid = ac.contratoid
left outer join cntcar_work.vw_redshift_ciclodevida_ativacao a on a.adesaoid = ac.adesaoid
left outer join cntcar_work.vw_ciclodevida_portifolio p on p.adesaoid = bh.adesaoid
where
upper(p.agregado) = 'PORTO SEGURO'
and upper(p.detalhado) not in ('CARROFACIL','CARRO FACIL','CARRO FÁCIL')
and upper(a.nomeplano) <> 'LOCADORA'
and TO_DATE(datahabilitada, 'YYYY-MM-01') = (SELECT MesRefVar FROM #MesRef_09)
;
DROP TABLE IF EXISTS #flagPlacasasPorto_09;
CREATE TABLE #flagPlacasasPorto_09 (documento VARCHAR, flagPlacas FLOAT);
INSERT INTO #flagPlacasasPorto_09
select
documento,
count(documento) as flagPlacas
from #baseHabilitada_09Porto
group by documento
having flagPlacas > 1
;
DROP TABLE IF EXISTS #baseHabilitada_09PFCom1Porto;
CREATE TABLE #baseHabilitada_09PFCom1Porto (documento VARCHAR, anomes DATE, agregado VARCHAR, anomescancelamento DATE);
INSERT INTO #baseHabilitada_09PFCom1Porto
select
a.documento::float,
a.anomes,
a.agregado,
a.anomescancelamento
from #baseHabilitada_09Porto a
left join #flagPlacasasPorto_09 b on a.documento = b.documento
where b.flagPlacas is null and len(a.documento) <=11
;
DROP TABLE IF EXISTS #basefinal_09;
CREATE TABLE #basefinal_09 (documento VARCHAR, anomes DATE, agregado VARCHAR, anomescancelamento DATE);
INSERT INTO #basefinal_09
select * from #baseHabilitada_09PFCom1PlacaSemPorto
union
select * from #baseHabilitada_09PFCom1Porto
;
select distinct * from #basefinal_09
"""

In [11]:
BasePF_Clientes1Placa_09 = sqlio.read_sql_query(cte_BasePF_Clientes1Placa_09, conn) 

c:\Users\raul.correa\Anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [12]:
print(BasePF_Clientes1Placa_09)

           documento      anomes      agregado anomescancelamento
0        35986213882  2022-06-01        Básico               None
1        35954427844  2022-06-01      COMPLETO               None
2        88254739404  2022-06-01          Itaú         2023-02-01
3        25834369880  2022-06-01        Básico               None
4        22723974847  2022-06-01      COMPLETO               None
...              ...         ...           ...                ...
1232405  39708582832  2022-06-01  Porto Seguro               None
1232406  34253175805  2022-06-01  Porto Seguro               None
1232407  93202490591  2022-06-01  Porto Seguro               None
1232408  37318952854  2022-06-01  Porto Seguro               None
1232409   8615412430  2022-06-01  Porto Seguro         2022-11-01

[1232410 rows x 4 columns]


In [ ]:
excluindo_09 = """
DROP TABLE IF EXISTS #MesRef_09;
DROP TABLE IF EXISTS #baseHabilitada_09;
DROP TABLE IF EXISTS #flagPlacasas_09;
DROP TABLE IF EXISTS #baseHabilitada_09PFCom1Porto;
DROP TABLE IF EXISTS #baseHabilitada_09Porto;
DROP TABLE IF EXISTS #baseHabilitada_09PFCom1PlacaSemPorto;
"""

In [ ]:
sqlio.read_sql_query(excluindo_09, conn) 

**Cruzando os meses possuindo como referencia o primeiro mes**

In [ ]:
cte_BasePF_Clientes1Placa_09 = sqlio.read_sql_query(cte_BasePF_Clientes1Placa_09, conn) 
cte_BasePF_Clientes1Placa_08 = sqlio.read_sql_query(cte_BasePF_Clientes1Placa_08, conn) 
cte_BasePF_Clientes1Placa_07 = sqlio.read_sql_query(cte_BasePF_Clientes1Placa_07, conn) 
cte_BasePF_Clientes1Placa_06 = sqlio.read_sql_query(cte_BasePF_Clientes1Placa_06, conn) 
cte_BasePF_Clientes1Placa_05 = sqlio.read_sql_query(cte_BasePF_Clientes1Placa_05, conn) 
cte_BasePF_Clientes1Placa_04 = sqlio.read_sql_query(cte_BasePF_Clientes1Placa_04, conn) 
cte_BasePF_Clientes1Placa_03 = sqlio.read_sql_query(cte_BasePF_Clientes1Placa_03, conn) 
cte_BasePF_Clientes1Placa_02 = sqlio.read_sql_query(cte_BasePF_Clientes1Placa_02, conn)
cte_BasePF_Clientes1Placa_01 = sqlio.read_sql_query(cte_BasePF_Clientes1Placa_01, conn)  
cte_BasePF_Clientes1Placa_00 = sqlio.read_sql_query(cte_BasePF_Clientes1Placa_00, conn)  

ConsolidaMeses = sqlio.read_sql_query(consolidando_meses, conn) 